In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


- 데이터 경로 변수

In [6]:
DATA_PATH = "/content/drive/MyDrive/파이널프로젝트/data/"
DATA_PATH

'/content/drive/MyDrive/파이널프로젝트/data/'

- 시드값

In [7]:
SEED = 42

- 데이터 불러오기

In [8]:
import pandas as pd
import numpy as np

train_tr = pd.read_csv(f"{DATA_PATH}store_train_transactions.csv") # 학습용 구매기록 데이터
train_target = pd.read_csv(f"{DATA_PATH}store_train.csv") # 학습용 정답 데이터
test_tr = pd.read_csv(f"{DATA_PATH}store_test_transactions.csv") # 테스트용 구매기록 데이터
submit = pd.read_csv(f"{DATA_PATH}store_submission.csv") # 제출 양식 데이터

train_tr.shape , train_target.shape , test_tr.shape , submit.shape

((523105, 7), (14940, 2), (441196, 7), (12225, 2))

- 공통 피처 파일 불러오기

In [9]:
train_ft = pd.read_csv(f"{DATA_PATH}train_common.csv") # 학습 데이터(피처)
test_ft = pd.read_csv(f"{DATA_PATH}test_common.csv") # 테스트 데이터(피처)

train_ft.shape , test_ft.shape

((14940, 329), (12225, 329))

# 결측치 처리

In [10]:
mask = train_ft.isnull().sum() > 0
train_ft.isnull().sum()[mask]

,0
구매금액표준편차,388


In [11]:
mask = test_ft.isnull().sum() > 0
test_ft.isnull().sum()[mask]

,0
구매금액표준편차,242


In [12]:
train_ft["구매금액표준편차"] = train_ft["구매금액표준편차"].fillna(0)
test_ft["구매금액표준편차"] = test_ft["구매금액표준편차"].fillna(0)

# 특성 공학(Feature Engineering)

- ID 변수 제외

In [13]:
train_ft = train_ft.iloc[:,1:]
test_ft = test_ft.iloc[:,1:]
train_ft.shape, test_ft.shape

((14940, 328), (12225, 328))

- 추가 피처 만들어 보기

In [14]:
cols = [ col for col in train_ft.columns if col.startswith("pivot_cnt_") ]

In [15]:
train_ft["중분류별_구매횟수_std"] = train_ft[cols].std(axis=1)
train_ft["중분류별_구매횟수_skew"] = train_ft[cols].skew(axis=1)
train_ft["중분류별_구매횟수_kurt"] = train_ft[cols].kurt(axis=1)

test_ft["중분류별_구매횟수_std"] = test_ft[cols].std(axis=1)
test_ft["중분류별_구매횟수_skew"] = test_ft[cols].skew(axis=1)
test_ft["중분류별_구매횟수_kurt"] = test_ft[cols].kurt(axis=1)

train_ft.shape, test_ft.shape

((14940, 331), (12225, 331))

## Feature Encoding

In [16]:
cols = train_ft.select_dtypes("object").columns.tolist()
train_ft[cols].nunique()

,0
주구매지점,4
주구매_중분류,246
주구매_대분류,28


In [17]:
%pip install category_encoders

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.0/82.0 kB 277.8 kB/s eta 0:00:00


In [18]:
import category_encoders as ce

In [19]:
enc = ce.one_hot.OneHotEncoder()
tmp = enc.fit_transform(train_ft[["주구매지점","주구매_대분류"]])
train_ft = pd.concat([train_ft,tmp],axis =1)

tmp = enc.transform(test_ft[["주구매지점","주구매_대분류"]])
test_ft = pd.concat([test_ft,tmp],axis =1)

train_ft.shape, test_ft.shape

((14940, 363), (12225, 363))

In [20]:
enc = ce.count.CountEncoder()
train_ft["주구매_중분류_cnt"] = enc.fit_transform(train_ft[["주구매_중분류"]])
test_ft["주구매_중분류_cnt"] = enc.transform(test_ft[["주구매_중분류"]])

train_ft.shape, test_ft.shape

((14940, 364), (12225, 364))

- 문자열 피처 삭제

In [21]:
cols

['주구매지점', '주구매_중분류', '주구매_대분류']

In [22]:
train_ft = train_ft.drop(columns=cols)
test_ft = test_ft.drop(columns=cols)
train_ft.shape, test_ft.shape

((14940, 361), (12225, 361))

In [23]:
train_ft.select_dtypes("object").columns , test_ft.select_dtypes("object").columns

(Index([], dtype='object'), Index([], dtype='object'))

## Feature Scaling

In [24]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [25]:
train_ft[train_ft.columns] = scaler.fit_transform(train_ft)
test_ft[test_ft.columns] = scaler.transform(test_ft)
train_ft.head()

,내점일수,구매주기,주말방문비율,봄_구매비율,여름_구매비율,가을_구매비율,겨울_구매비율,주구매요일,일별평균구매건수,거래개월수,...,주구매_대분류_20,주구매_대분류_21,주구매_대분류_22,주구매_대분류_23,주구매_대분류_24,주구매_대분류_25,주구매_대분류_26,주구매_대분류_27,주구매_대분류_28,주구매_중분류_cnt
0,-0.437126,0.002987,0.257728,-1.029777,0.001191,0.838272,0.338186,0.109631,-0.057297,0.017620,...,-0.081258,-0.18062,-0.171941,-0.124766,-0.074744,-0.08084,-0.089606,-0.101046,-0.048458,-0.726970
1,0.239394,-0.356452,-1.008554,0.323951,-0.390607,0.620171,-0.552996,0.109631,0.222706,1.176010,...,-0.081258,-0.18062,-0.171941,-0.124766,-0.074744,-0.08084,-0.089606,-0.101046,-0.048458,-0.510078
2,1.890101,-0.869935,0.036742,0.798943,-0.514333,-0.304527,-0.059266,-1.643370,0.277707,1.465608,...,-0.081258,-0.18062,-0.171941,-0.124766,-0.074744,-0.08084,-0.089606,-0.101046,-0.048458,-0.848408
3,4.298510,-1.023980,-0.080558,0.420933,-0.327474,0.008592,-0.135636,0.109631,0.674668,1.465608,...,-0.081258,-0.18062,-0.171941,-0.124766,-0.074744,-0.08084,-0.089606,-0.101046,-0.048458,1.523851
4,0.618244,-0.613193,0.302875,-0.752532,1.707410,-0.130285,-0.821561,0.693965,0.679008,0.886413,...,-0.081258,-0.18062,-0.171941,-0.124766,-0.074744,-0.08084,-0.089606,-0.101046,-0.048458,-0.833157


# 정답 데이터

In [26]:
target = train_target["target"]
target

,target
0,1.0
1,1.0
2,0.0
3,0.0
4,0.0
...,...
14935,0.0
14936,0.0
14937,0.0
14938,1.0


# cv 점수 확인해보기

In [27]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
cv = KFold(n_splits=5,shuffle=True, random_state=SEED)

In [28]:
from lightgbm import LGBMClassifier

model = LGBMClassifier(random_state=SEED)
scores = cross_val_score(model,train_ft,target,cv = cv ,scoring='f1_macro',n_jobs = -1)
np.mean(scores)

/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


0.6944709437561055

# 모델 학습

In [29]:
model = LGBMClassifier(random_state=SEED)
model.fit(train_ft,target)

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 5874, number of negative: 9066
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.152828 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 8130
[LightGBM] [Info] Number of data points in the train set: 14940, number of used features: 345
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.393173 -> initscore=-0.433995
[LightGBM] [Info] Start training from score -0.433995


LGBMClassifier(random_state=42)

# 테스트 데이터 예측

In [30]:
pred = model.predict(test_ft)
pred

array([0., 1., 0., ..., 0., 0., 0.])

In [34]:
# pred = model.predict_proba(test_ft)[:,1]
# pred

array([0.20176087, 0.65097769, 0.45199027, ..., 0.05146371, 0.34686509,
       0.03600213])

# 평가를 위한 제출 파일 생성
- 예측 결과를 target 컬럼에 넣어 csv 파일로 저장후에 제출한다.

In [31]:
submit

,ID,target
0,test_0,0.5
1,test_1,0.5
2,test_2,0.5
3,test_3,0.5
4,test_4,0.5
...,...,...
12220,test_12220,0.5
12221,test_12221,0.5
12222,test_12222,0.5
12223,test_12223,0.5


In [35]:
submit["target"] = pred
submit

,ID,target
0,test_0,0.201761
1,test_1,0.650978
2,test_2,0.451990
3,test_3,0.704167
4,test_4,0.297416
...,...,...
12220,test_12220,0.566104
12221,test_12221,0.273405
12222,test_12222,0.051464
12223,test_12223,0.346865


- 예측 결과를 csv 파일로 저장하여 제출

In [33]:
submit.to_csv(f"{DATA_PATH}submit.csv",index=False)